## Load Libraries and Slice Name

In [ ]:
from state import *
from IPython.core.getipython import get_ipython

# Import libraries
%run ../setup/include/include_libraries.py
# Import selected slice name
%run ../slice_info/selected_slice.py
# Import topology_variables
# If you manually run the notebook, please specify the slice name. 
# Use the button in the GUI automatically does it for you
if os.getenv('SELECTED_SLICE') is not None:
    SELECTED_SLICE = os.getenv('SELECTED_SLICE')

# If you manually run the notebook, please specify the slice name in the line below
# SELECTED_SLICE = 'slice_name'
path = f'../slice_info/{SELECTED_SLICE}/topology_variables.ipynb'
get_ipython().run_line_magic('run', path)

In [ ]:
try:
    from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
    
    slice = fablib.get_slice(name=selected_slice)
    success = True

except Exception as e:
    print(f"Exception: {e}")
    success=False
    raise SystemExit("Stopping notebook execution due to exception")

## Configure Node1

In [ ]:
try:
    node1 = slice.get_node(name=node1_name)        
    node1_iface = node1.get_interface(network_name=network1_name)  #AB
    node1_iface.ip_addr_add(addr=node1_IPv4_addr0, subnet=subnet1)
    
    node2_iface = node1.get_interface(network_name=network3_name)  #AC
    node2_iface.ip_addr_add(addr=node1_IPv4_addr1, subnet=subnet3)
    
    stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')
    stdout, stderr = node1.execute(f'ip route list')
    success= True

except Exception as e:
    print(f"Exception: {e}")
    success = False
    raise SystemExit("Stopping notebook execution due to exception")

## Configure Node2

In [ ]:
try:
    node2 = slice.get_node(name=node2_name)        
    node2_iface = node2.get_interface(network_name=network1_name)  #AB
    node2_iface.ip_addr_add(addr=node2_IPv4_addr0, subnet=subnet1)
    
    node2_iface = node2.get_interface(network_name=network2_name)  #BC
    node2_iface.ip_addr_add(addr=node2_IPv4_addr1, subnet=subnet2)
    
    stdout, stderr = node2.execute(f'ip addr show {node2_iface.get_os_interface()}')    
    stdout, stderr = node2.execute(f'ip route list')
    success=True

except Exception as e:
    success=False
    print(f"Exception: {e}")
    raise SystemExit("Stopping notebook execution due to exception")

## Configure Node3

In [ ]:
try:
    node3 = slice.get_node(name=node3_name)        
    node3_iface = node3.get_interface(network_name=network2_name)  #BC
    node3_iface.ip_addr_add(addr=node3_IPv4_addr0, subnet=subnet2)
    
    node3_iface = node3.get_interface(network_name=network3_name)  #CA
    node3_iface.ip_addr_add(addr=node3_IPv4_addr1, subnet=subnet3)
 
    stdout, stderr = node3.execute(f'ip addr show {node3_iface.get_os_interface()}')    
    stdout, stderr = node3.execute(f'ip route list')
    success=True

except Exception as e:
    print(f"Exception: {e}")
    success= False
    raise SystemExit("Stopping notebook execution due to exception")

## Configure All nodes

In [ ]:
try:
    for node in slice.get_nodes():
        node.execute("sudo apt update")
        node.execute('sudo DEBIAN_FRONTEND=noninteractive apt-get install -y net-tools')
        node.execute("sudo apt install -y traceroute")
        
        for interface in node.get_interfaces():
            stdout, stderr = node.execute(f'sudo ip link set dev { interface.get_device_name()} up')
        print(f"Network status at node {node.get_name()}")
        node.execute("ip a")
    success=True
        
except Exception as e:
    print(f"Exception: {e}")
    success=False
    raise SystemExit("Stopping notebook execution due to exception")

## Update state

In [ ]:
if success:
    update_state_local(selected_slice, 'SLICE_CONFIGURED')
    update_state_in_fim(selected_slice, read_state_from_local(selected_slice))
else:
    print ('State is not updated due to errors')